In [10]:
!pip install tensorflow-datasets

  Retrying (Retry(total=4, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<pip._vendor.urllib3.connection.VerifiedHTTPSConnection object at 0x7efe55102860>: Failed to establish a new connection: [Errno 101] Network is unreachable',)': /simple/tensorflow-datasets/
  Retrying (Retry(total=3, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<pip._vendor.urllib3.connection.VerifiedHTTPSConnection object at 0x7efe55102f98>: Failed to establish a new connection: [Errno 101] Network is unreachable',)': /simple/tensorflow-datasets/
  Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<pip._vendor.urllib3.connection.VerifiedHTTPSConnection object at 0x7efe55102e48>: Failed to establish a new connection: [Errno 101] Network is unreachable',)': /simple/tensorflow-datasets/
  Retrying (Retry(total=1, connect=None, re

In [13]:
import tensorflow as tf
#import tensorflow_datasets as tfds
import math
import cifar10
import numpy as np
max_step=3000
batch_size=128
num_examples = 10000

ImportError: No module named 'tensorflow_datasets'

In [ ]:
# def _get_images_labels(batch_size, split, distords=False):
#     """Returns Dataset for given split."""
#     dataset = tfds.load(name='cifar10', split=split)
#     scope = 'data_augmentation' if distords else 'input'
#     with tf.name_scope(scope):
#         dataset = dataset.map(DataPreprocessor(distords), num_parallel_calls=10)
#     # Dataset is small enough to be fully loaded on memory:
#     dataset = dataset.prefetch(-1)
#     dataset = dataset.repeat().batch(batch_size)
#     iterator = dataset.make_one_shot_iterator()
#     images_labels = iterator.get_next()
#     images, labels = images_labels['input'], images_labels['target']
#     tf.summary.image('images', images)
#     return images, labels

# def distorted_inputs(batch_size):
#     return _get_images_labels(batch_size, tfds.Split.TRAIN, distords=True)

# def inputs(eval_data, batch_size):
#     split = tfds.Split.TEST if eval_data == 'test' else tfds.Split.TRAIN
#     return _get_images_labels(batch_size, split)

def variable_with_weight_loss(shape,stddev,w):
    var = tf.Variable(tf.truncated_normal(shape,stddev=stddev))
    if w is not None:
        weight_loss = tf.multiply(tf.nn.l2_loss(var),w,name='weight_loss')
        tf.add_to_collection('losses',weight_loss)
    return var

def loss(logits,labels):
    labels = tf.cast(labels,tf.int64)
    cross_entropy = tf.nn.sparse.softmax_cross_entropy_with_logits(logits=logits,labels=labels,name='cross_entropy_per_example')
    cross_entropy_mean = tf.reduce_mean(cross_entropy,name='cross_entropy')
    tf.add_to_collection('losses',cross_entropy_mean)
    return tf.add_n(tf.get_collection('losses'),name='total_loss')

In [ ]:
#net define
images_train,labels_train = cifar10.distorted_inputs(batch_size)
images_test,labels_test = cifar10.inputs(eval_data=True,batch_size=batch_size)

image_holder = tf.placeholder(tf.float32,shape=[batch_size,24,24,3])
label_holder = tf.placeholder(tf.float32,shape=[batch_size])

w1 = variable_with_weight_loss(shape=[5,5,3,64],stddev=5e-2,w=0.0)
b1 = tf.Variable(tf.constant(0.0,shape=[64]))
h1 = tf.nn.relu(tf.nn.bias_add(tf.nn.conv2d(image_holder,w1,[1,1,1,1],padding='SAME'),b1))
h1_pool = tf.nn.max_pool(h1,ksize=[1,3,3,1],strides=[1,2,2,1],padding='SAME')
norml = tf.nn.lrn(h1_pool,4,bias=1.0,padding=0.001/9.0,beta=0.75)

w2 = variable_with_weight_loss(shape=[5,5,64,64],stddev=5e-2,w=0.0)
b2 = tf.Variable(tf.constant(0.1,shape=[64]))
h2 = tf.nn.relu(tf.nn.bias_add(tf.nn.conv2d(norml,w2,[1,1,1,1],padding='SAME'),b2))
norm2 = tf.nn.lrn(h2,4,bias=1.0,padding=0.001/9.0,beta=0.75)
h2_pool = tf.nn.max_pool(norm2,ksize=[1,3,3,1],strides=[1,2,2,1],padding='SAME')

reshape = tf.reshape(h2_pool,[batch_size,-1])
dim = reshape.get_shape()[1].value
w3 = variable_with_weight_loss(shape=[dim,384],stddev=0.04,w=0.004)
b3 = tf.Variable(tf.constant(0.1,shape=[384]))
fc3 = tf.nn.relu(tf.matmul(reshape,w3)+b3)

w4 = variable_with_weight_loss(shape=[384,192],stddev=0.04,w=0.004)
b4 = tf.Variable(tf.constant(0.1,shape=[192]))
fc4 = tf.nn.relu(tf.matmul(fc3,w4)+b4)

w5 = variable_with_weight_loss(shape=[192,10],stddev=1/192.0,w=0.0)
b5 = tf.Variable(tf.constant(0.1,shape=[10]))
logits = tf.add(tf.matmul(fc4,w5),b5)

loss = loss(logits,label_holder)
train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)

saver=tf.train.Saver()

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    tf.train.start_queue_runners()
    for step in range(max_steps):
        image_batch,label_batch = sess.run([images_train,labels_train])
        _,loss_value = sess.run([train_op,loss],{image_holder:image_batch,label_holder:label_batch})
        if step % 10==0:
            print('step %d,loss value %f'%(step,loss_value))
    saver.save(sess,'cifar.ckpt')

In [6]:
num_iter = int(math.ceil(num_examples/batch_size))
true_count=0
total_sample_count = num_iter*batch_size
step=0
while step<num_iter:
    image_batch,label_batch = sess.run([images_test,labels_test])
    predictions = sess.run([top_k_op],{image_holder:image_batch,label_holder:label_batch})
    true_count +=np.sum(predictions)
    step+=1
precision = true_count/total_sample_count
print('precision:',"{:.3f}".format(precision))

NameError: name 'math' is not defined